In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("..")

import os
import optuna
import torch
import wandb

from src.training import train_model, build_dataset
from src.model_unet import UNet

# Path to Dataset
DATA_DIR = "/media/gabriel/500 BKP/datasets/SNSD-DataSet/preprocessed"
CHECKPOINTS_DIR = os.path.dirname(os.getcwd()) + "/checkpoints"


/home/gabriel/miniconda3/envs/venv_audio/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# === Optuna ===
def objective(trial):

    # === Hyperparameter search space ===
    base_filters = trial.suggest_categorical('base_filters', [16, 32, 64])
    depth = trial.suggest_int('depth', 3, 5)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.3)
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 24])

    # === Init wandb logging ===
    wandb.init(project='audio-denoising', config={
        'base_filters': base_filters,
        'depth': depth,
        'dropout_rate': dropout_rate,
        'lr': lr,
        'batch_size': batch_size
    }, reinit=True)

    # === Build data ===
    train_dataset, train_loader = build_dataset(DATA_DIR, split='train', batch_size=batch_size)
    val_dataset, val_loader = build_dataset(DATA_DIR, split='validation', batch_size=batch_size)

    # === Build model ===
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UNet(in_channels=1, out_channels=1, base_filters=base_filters, depth=depth, dropout_rate=dropout_rate)
    model = model.to(device)

    # === Optimizer and loss ===
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2)
    criterion = torch.nn.MSELoss()

    # === Train ===
    model, train_losses, val_losses = train_model(
        model, optimizer, criterion, scheduler,
        device='cuda',
        train_loader=train_loader,
        train_dataset=train_dataset,
        val_loader=val_loader,
        val_dataset=val_dataset,
        num_epochs=50,
        early_stopping_patience=5,
        audio_preview=False,
        trial=trial,
        save_dir=CHECKPOINTS_DIR
    )

    # === Final value to minimize ===
    return val_losses[-1]

In [4]:
study = optuna.create_study(
    study_name="audio-denoising",
    direction="minimize",
    storage="sqlite:///optuna_trials.db",
    load_if_exists=False,
)
study.optimize(objective, n_trials=50, timeout=8*60*60)  # 8 hours

[I 2025-04-20 21:41:49,954] A new study created in RDB with name: audio-denoising
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: gfiuzaa (gfiuzaa-universidade-federal-do-rio-de-janeiro). Use `wandb login --relogin` to force relogin



🔁 Epoch 1/50


[W 2025-04-20 21:41:54,518] Trial 0 failed with parameters: {'base_filters': 32, 'depth': 4, 'dropout_rate': 0.06853158491121444, 'lr': 0.00010787715079229057, 'batch_size': 16} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 258.00 MiB (GPU 0; 7.79 GiB total capacity; 7.17 GiB already allocated; 167.94 MiB free; 7.37 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF').
Traceback (most recent call last):
  File "/home/gabriel/miniconda3/envs/venv_audio/lib/python3.7/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_11229/635556042.py", line 46, in objective
    save_dir=CHECKPOINTS_DIR
  File "../src/training.py", line 90, in train_model
    avg_train_loss = run_epoch(model, train_loader, criterion, device, optimizer, scaler, is

OutOfMemoryError: CUDA out of memory. Tried to allocate 258.00 MiB (GPU 0; 7.79 GiB total capacity; 7.17 GiB already allocated; 167.94 MiB free; 7.37 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [ ]:
import optuna
import pandas as pd

def export_study_to_csv(study_path, study_name, out_csv="optuna_trials.csv"):
    study = optuna.load_study(study_name=study_name, storage=study_path)

    rows = []
    for trial in study.trials:
        row = {
            "trial": trial.number,
            "val_loss": trial.value,
            "model_path": trial.user_attrs.get("best_model_path", "N/A")
        }
        # Add hyperparameters
        row.update(trial.params)
        rows.append(row)

    df = pd.DataFrame(rows)
    df = df.sort_values("val_loss")
    df.to_csv(out_csv, index=False)

    print(f"✅ Exported {len(df)} trials to {out_csv}")

In [ ]:
export_study_to_csv(
    study_path="sqlite:///optuna_trials.db",
    study_name="audio-denoising",
    out_csv=os.path.dirname(os.getcwd()) + "/tuning/optuna_trials.csv"
)